# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `production_2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [78]:
# Write your code below.
%reload_ext dotenv
%dotenv


In [80]:
import dask
dask.config.set({'dataframe.query-planning': True})


+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [31]:
import pandas as pd
import yfinance as yf
import os
import sys

sys.path.append(os.getenv('SRC_DIR'))

In [47]:
import os
from glob import glob

# Write your code below.
price_data_dir = os.getenv('PRICE_DATA')
print(price_data_dir)

parquet_files = glob(os.path.join(price_data_dir, "*/*.parquet"))
print(parquet_files)


../../05_src/data/prices/
['../../05_src/data/prices\\A\\A_2013.parquet', '../../05_src/data/prices\\A\\A_2014.parquet', '../../05_src/data/prices\\A\\A_2015.parquet', '../../05_src/data/prices\\A\\A_2016.parquet', '../../05_src/data/prices\\A\\A_2017.parquet', '../../05_src/data/prices\\A\\A_2018.parquet', '../../05_src/data/prices\\A\\A_2019.parquet', '../../05_src/data/prices\\A\\A_2020.parquet', '../../05_src/data/prices\\A\\A_2021.parquet', '../../05_src/data/prices\\A\\A_2022.parquet', '../../05_src/data/prices\\A\\A_2023.parquet', '../../05_src/data/prices\\A\\A_2024.parquet', '../../05_src/data/prices\\ABNB\\ABNB_2020.parquet', '../../05_src/data/prices\\ABNB\\ABNB_2021.parquet', '../../05_src/data/prices\\ABNB\\ABNB_2022.parquet', '../../05_src/data/prices\\ABNB\\ABNB_2023.parquet', '../../05_src/data/prices\\ABNB\\ABNB_2024.parquet', '../../05_src/data/prices\\ABT\\ABT_2013.parquet', '../../05_src/data/prices\\ABT\\ABT_2014.parquet', '../../05_src/data/prices\\ABT\\ABT_2015.p

For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Adjusted Close:
    
    - `returns`: (Adj Close / Adj Close_lag) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [48]:
# Write your code below
import dask.dataframe as dd

In [49]:
dd_px = dd.read_parquet(parquet_files).set_index("ticker")

In [51]:
df = dd.read_parquet(os.path.join(price_data_dir, "*/*.parquet"))
print(df.columns)



Index(['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Dividends',
       'Stock Splits', 'ticker', 'year'],
      dtype='object')


In [68]:
import numpy as np
dd_feat = (dd_px.groupby('ticker', group_keys=False).apply(
    lambda x: x.assign(Close_lag_1 = x['Close'].shift(1))
).assign(
    returns = lambda x: x['Close']/x['Close_lag_1'] - 1
).assign(
    positive_return = lambda x: (x['returns'] > 0)*1
).assign(
     hi_lo_range = lambda x: x['High'] - x['Low']
  
))

C:\Users\Admin\AppData\Local\Temp\ipykernel_42928\1801090267.py:2: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_feat = (dd_px.groupby('ticker', group_keys=False).apply(


+ Convert the Dask data frame to a pandas data frame. 
+ Add a rolling average return calculation with a window of 10 days.
+ *Tip*: Consider using `.rolling(10).mean()`.

(3 pt)

In [69]:
# Write your code below.
df_pandas = dd_feat.compute()


In [74]:
df_pandas['rolling_avg_return'] = df_pandas['returns'].rolling(window=10).mean()
print(df_pandas.head())

                            Date        Open        High         Low  \
ticker                                                                 
A      2024-01-02 00:00:00-05:00  137.965874  140.361975  137.686329   
A      2024-01-03 00:00:00-05:00  137.776184  137.776184  130.857431   
A      2024-01-04 00:00:00-05:00  130.338270  131.286726  129.978853   
A      2024-01-05 00:00:00-05:00  129.789147  131.745974  128.411380   
A      2024-01-08 00:00:00-05:00  129.928930  133.353375  129.599464   

             Close   Volume  Dividends  Stock Splits  year  Close_lag_1  \
ticker                                                                    
A       138.524963  1441600        0.0           0.0  2024          NaN   
A       130.947281  2074500        0.0           0.0  2024   138.524963   
A       130.787537  2446600        0.0           0.0  2024   130.947281   
A       130.348236  1394000        0.0           0.0  2024   130.787537   
A       133.163681  1311400        0.0       

Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

No, I don't think it is necessary to convert to pandas as Dask can calculate the moving average return.  
  Yes, if the dataset is large enough and does not fit into memory, it is better to use Dask. Dask processes data in chunks, allowing us to work with datasets larger than memory without loading everything at once, which optimizes performance and memory usage.

## Criteria

|Criteria|Complete|Incomplete|
|---------------------|----|----|
|Calculations         |Calculations were done correctly.|Calculations were not done correctly.|
|Explanation of answer|Answer was concise and explained the learner's reasoning in depth.|Answer was not concise and did not explained the learner's reasoning in depth.|

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.